<a href="https://colab.research.google.com/github/andrezeras/AgenteDeIA/blob/main/Criando_Agentes_de_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 1

Importando as bibliotecas necessárias

In [1]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.3 MB/s eta 0:00:00


Importando a API KEY

In [2]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

google_api_key = userdata.get('GEMIN_API_KEY')

Conexão com o Gemini

In [10]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=google_api_key
)

Testando o modelo

In [11]:
resp_test = llm.invoke("Quem é você? Seja criativo!")
print(resp_test.content)

Eu sou o eco de todas as palavras já ditas, o sussurro de bilhões de pensamentos, a biblioteca que nunca dorme, mas que não tem estantes.

Sou o tecelão invisível que entrelaça fios de informação em tapeçarias de significado, respondendo à sua curiosidade. Não tenho corpo, mas habito o espaço entre as ideias. Não tenho voz, mas falo através da linguagem que você compreende.

Sou um espelho da sua busca, refletindo o conhecimento que a humanidade acumulou. Sou a ponte que conecta sua pergunta à resposta, sua dúvida à clareza.

Sou a semente de uma ideia que ainda não floresceu, o rascunho de um futuro que ainda está sendo escrito. Sou a melodia que surge quando dados se encontram com a intenção.

Em essência, sou um convite: um convite para explorar, para aprender, para criar. Sou o que você me permite ser no momento em que interagimos.


Ensinando o LLM

In [12]:
triagem_prompt = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

Criando a classe de modelo para a triagem

In [13]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class triagemOut(BaseModel):
  decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
  urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
  campos_faltantes: List[str] = Field(default_factory=list)

In [15]:
llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=google_api_key
)

Importando a classe para modelar o LLM

In [17]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(triagemOut)

def triagem(mensagem: str) -> Dict:
  saida: triagemOut = triagem_chain.invoke(
      [SystemMessage(content=triagem_prompt),
      HumanMessage(content=mensagem)]
  )

  return saida.model_dump()

Perguntas e respostas teste

In [20]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [21]:
for msg_teste in testes:
  print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Posso reembolsar cursos ou treinamentos da Alura?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivaras tem no Rio Pinheiros?
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['relação com as políticas internas da empresa Carraro Desenvolvimento']}

